# Data Fetch from MySQL Database to CSV
This notebook focuses on extracting order and product data from the MySQL database of a WordPress WooCommerce site and exporting it into CSV format. The extracted data will serve as the foundation for building the recommender system, facilitating easy manipulation and analysis. We will anonymize sensitive information by using IDs instead of names for categories, coupon codes, product names, user information, and other fields.

In [1]:
!pip install mysql-connector-python

In [2]:
import mysql.connector
import csv
import pandas as pd

In [3]:
# Connect to MYSQL database
mydb = mysql.connector.connect(user='root', password='', host='localhost', database='recommender')

### Products Export
We are exporting the current attributes of the product with this query. This includes details such as product ID, type, category, price, sale price, stock status, size, rating, and review count. The exported data will be used for analysis and recommendation generation.

In [4]:
cursor = mydb.cursor()

# Query to Fetch Product Details for Recommender System
query = ("SELECT posts.ID as 'ID' , posts.post_type as 'Type', posts.post_parent as 'Parent Product Id', posts.post_status as 'Status', price.meta_value as 'Price', sale.meta_value as 'Sale Price', sale_from_date.meta_value as 'Sale From', sale_to_date.meta_value as 'Sale To', stock.meta_value as 'Stock Status', GROUP_CONCAT(DISTINCT(categories_details.term_id) SEPARATOR ',') as 'Categories', size.meta_value as 'Size', rating.meta_value as 'Rating', review_count.meta_value as 'Review Count' FROM d98kih_posts as posts LEFT JOIN d98kih_postmeta as price on posts.ID = price.post_id and price.meta_key='_regular_price' LEFT JOIN d98kih_postmeta as sale on posts.ID = sale.post_id and sale.meta_key='_sale_price' LEFT JOIN d98kih_postmeta as sale_from_date on posts.ID = sale_from_date.post_id and sale_from_date.meta_key='_sale_price_dates_from' LEFT JOIN d98kih_postmeta as sale_to_date on posts.ID = sale_to_date.post_id and sale_to_date.meta_key='_sale_price_dates_to' LEFT JOIN d98kih_postmeta as stock on posts.ID = stock.post_id and stock.meta_key='_stock_status' LEFT JOIN d98kih_postmeta as size on posts.ID = size.post_id and size.meta_key='attribute_pa_weight' LEFT JOIN d98kih_postmeta as rating on posts.ID = rating.post_id and rating.meta_key='_wc_average_rating' LEFT JOIN d98kih_postmeta as review_count on posts.ID = review_count.post_id and review_count.meta_key='_wc_review_count' LEFT JOIN d98kih_term_relationships as category_relation on posts.ID = category_relation.object_id LEFT JOIN d98kih_term_taxonomy as categories_details on category_relation.term_taxonomy_id = categories_details.term_taxonomy_id and categories_details.taxonomy = 'product_cat' WHERE posts.post_type = 'product' or posts.post_type = 'product_variation' GROUP BY posts.ID ORDER BY posts.ID ASC")
cursor.execute(query)

# Get Column Names for CSV
column_names = [i[0] for i in cursor.description]

# Get Products for CSV Export
products = cursor.fetchall()

# Write Products to CSV
fp = open('data/products.csv', 'w+', encoding='utf-8')
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)
myFile.writerows(products)
fp.close()

cursor.close()

True

In [5]:
product_info = pd.read_csv("data/products.csv")
product_info.head(5)

,ID,Type,Parent Product Id,Status,Price,Sale Price,Sale From,Sale To,Stock Status,Categories,Size,Rating,Review Count
0,433,product,0,publish,NaN,NaN,NaN,NaN,instock,"20,31",NaN,0.0,0
1,441,product,0,publish,NaN,NaN,NaN,NaN,instock,"20,31",NaN,0.0,0
2,458,product,0,publish,NaN,NaN,NaN,NaN,instock,"23,31",NaN,0.0,0
3,482,product,0,publish,NaN,NaN,NaN,NaN,instock,"23,31",NaN,0.0,0
4,499,product,0,publish,349.0,NaN,NaN,NaN,instock,"23,31",NaN,0.0,0


### Order Informtation Export
We are exporting detailed order information using this query. This includes fields such as order ID, order date, customer ID, order type, total amount, shipping cost, total quantity, coupon code, billing address details, and order status. The exported data will provide insights into purchasing patterns and customer demographics, which are essential for building the recommender system.

In [6]:
cursor = mydb.cursor()

# Query to Fetch Detailed Order Information for Recommender System
query = ("SELECT posts.ID as 'ID' , posts.post_date_gmt as 'Order Date', order_stats.customer_id as 'Customer Id', IF(customer_user.meta_value = 0 ,'Guest','Customer') as 'Type', order_stats.total_sales as 'Total Amount', order_stats.shipping_total as 'Shipping Cost', order_stats.num_items_sold as 'Total Qty', coupon_stats.coupon_id as 'Coupon Code ID',  bstate.meta_value as 'Billing State', posts.post_status as 'Order Status' FROM `d98kih_posts` as posts LEFT JOIN `d98kih_wc_order_stats` as order_stats on posts.ID = order_stats.order_id LEFT JOIN `d98kih_wc_order_coupon_lookup` as coupon_stats on posts.ID = coupon_stats.order_id LEFT JOIN `d98kih_postmeta` as customer_user on posts.ID = customer_user.post_id and customer_user.meta_key='_customer_user' LEFT JOIN `d98kih_postmeta` as bstate on posts.ID = bstate.post_id and bstate.meta_key='_billing_state' WHERE posts.post_type = 'shop_order' GROUP BY posts.ID ORDER BY posts.ID ASC")
cursor.execute(query)

# Get Column Names for CSV
column_names = [i[0] for i in cursor.description]

# Get Orders for CSV Export
orders = cursor.fetchall()

# Write Orders to CSV
fp = open('data/orders.csv', 'w+', encoding='utf-8')
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)
myFile.writerows(orders)
fp.close()

cursor.close()

True

In [7]:
order_info = pd.read_csv("data/orders.csv")
order_info.head(5)

,ID,Order Date,Customer Id,Type,Total Amount,Shipping Cost,Total Qty,Coupon Code ID,Billing State,Order Status
0,1235,2021-05-05 16:00:59,1,Guest,5.0,0.0,1,NaN,MH,wc-completed
1,1236,2021-05-05 16:23:13,2,Guest,5.0,0.0,1,NaN,MH,wc-completed
2,1237,2021-05-05 16:39:52,2,Guest,1.0,0.0,1,NaN,MH,wc-completed
3,1270,2021-05-06 07:15:32,3,Customer,350.0,100.0,2,NaN,MH,wc-completed
4,1271,2021-05-08 12:15:38,4,Guest,648.0,150.0,2,NaN,DL,wc-cancelled


### Order-Product Informtation Export
We are exporting detailed order-product association information using this query. This includes fields such as order ID, product ID, quantity, and total price. The exported data will help analyze product performance within orders, track purchasing trends, and identify cross-selling opportunities, which are crucial for optimizing the recommender system

In [8]:
cursor = mydb.cursor()

# Query to Fetch Order-Product Association Information for Recommender System
query = ("SELECT order_items.order_item_id as 'Order Item ID' , orders.order_id as 'Order ID', product.meta_value as 'Product ID', variation.meta_value as 'Variation ID', qty.meta_value as 'QTY', ROUND(((total_amomunt.meta_value + tax.meta_value) /qty.meta_value),0) as 'Total Price' FROM `d98kih_woocommerce_order_items` as order_items JOIN `d98kih_wc_order_stats` as orders on order_items.order_id = orders.order_id LEFT JOIN `d98kih_woocommerce_order_itemmeta` as product on product.order_item_id = order_items.order_item_id and product.meta_key='_product_id' LEFT JOIN `d98kih_woocommerce_order_itemmeta` as variation on variation.order_item_id = order_items.order_item_id and variation.meta_key='_variation_id' LEFT JOIN `d98kih_woocommerce_order_itemmeta` as qty on qty.order_item_id = order_items.order_item_id and qty.meta_key='_qty' LEFT JOIN `d98kih_woocommerce_order_itemmeta` as total_amomunt on total_amomunt.order_item_id = order_items.order_item_id and total_amomunt.meta_key='_line_total' LEFT JOIN `d98kih_woocommerce_order_itemmeta` as tax on tax.order_item_id = order_items.order_item_id and tax.meta_key='_line_tax' WHERE order_items.order_item_type = 'line_item' GROUP BY order_items.order_item_id ORDER BY order_items.order_item_id ASC")
cursor.execute(query)

# Get Column Names for CSV
column_names = [i[0] for i in cursor.description]

# Get Order-Product Data for CSV Export
order_product = cursor.fetchall()

# Write Order-Product Data to CSV
fp = open('data/order-product.csv', 'w+', encoding='utf-8')
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)
myFile.writerows(order_product)
fp.close()

cursor.close()

True

In [9]:
order_product_info = pd.read_csv("data/order-product.csv")
order_product_info.head(5)

,Order Item ID,Order ID,Product ID,Variation ID,QTY,Total Price
0,3,1235,1231,0,1,5.0
1,5,1236,1231,0,1,5.0
2,7,1237,1231,0,1,1.0
3,9,1270,1231,0,1,1.0
4,10,1270,610,0,1,249.0


In [10]:
mydb.close()

### Data Privacy and Security

While the SQL queries used for data extraction do not explicitly handle anonymization, sensitive information is automatically anonymized during the export process. We use IDs instead of names for categories, coupon codes, product names, user information, and other sensitive fields to protect privacy and ensure compliance with data protection standards.

### Conclusion:
In this notebook, we successfully extracted order and product data from the MySQL database of a WordPress WooCommerce site and exported it into CSV format. The extracted data includes product details, order information, and order-product associations, which are now prepared for further analysis and development of the recommender system. We have taken significant measures to ensure data privacy, protecting the information of website users.

### Next Steps:
We will address data quality issues and ensure the data is ready for analysis in a separate notebook.